# [Single-cell RNA-seq reveals cell type–specific molecular and genetic associations to lupus](https://pubmed.ncbi.nlm.nih.gov/35389781/)
GEO: [GSE174188](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE174188)

In [4]:
suppressPackageStartupMessages({
    library(Seurat)
    library(celldex)
    library(SingleR)
    library(ShinyCell)
    library(harmony)
    library(glmGamPoi)
    library(anndata)
    })

In [5]:
system("ulimit -m 500000000")

In [4]:
# Read in data
anndata_object <- read_h5ad("data/GSE174188_CLUES1_adjusted.h5ad")

# Create index to split data in half
half_index <- ceiling(nrow(anndata_object) / 2)

# Split data
adata1 <- anndata_object[1:half_index, ]
write_h5ad(adata1, "data/file1.h5ad")

adata2 <- anndata_object[(half_index + 1):nrow(anndata_object), ]
write_h5ad(adata2, "data/file2.h5ad")

In [6]:
# Create Seurat object from first half
part1 <- read_h5ad("data/file1.h5ad")
part1 <- CreateSeuratObject(counts = t(part1$X), meta.data = part1$obs)
part1
save(part1, file = file.path("object", "part1.rda"));rm(part1)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


An object of class Seurat 
1999 features across 631838 samples within 1 assay 
Active assay: RNA (1999 features, 0 variable features)
 1 layer present: counts

In [7]:
# Create Seurat object from second half
part2 <- read_h5ad("data/file2.h5ad")
part2 <- CreateSeuratObject(counts = t(part2$X), meta.data = part2$obs)
part2
save(part2, file = file.path("object", "part2.rda"));rm(part2)

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”


An object of class Seurat 
1999 features across 631838 samples within 1 assay 
Active assay: RNA (1999 features, 0 variable features)
 1 layer present: counts

In [8]:
# Merge data
load(file = file.path("object", "part1.rda"));load(file = file.path("object", "part2.rda"))
obj <- merge(x = part1, y = part2);rm(part1, part2)
obj
save(obj, file = file.path("object", "00.rda"))

An object of class Seurat 
1999 features across 1263676 samples within 1 assay 
Active assay: RNA (1999 features, 0 variable features)
 2 layers present: counts.1, counts.2

In [9]:
head(obj@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,batch_cov,ind_cov,Processing_Cohort,louvain,cg_cov,ct_cov,L3,ind_cov_batch_cov,Age,Sex,pop_cov,Status,SLE_status
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
CAAGGCCAGTATCGAA-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0,SeuratProject,-91.619885,75,dmx_YS-JY-22_pool6,HC-546,4.0,1,T4,T4_naive,1.0,HC-546:dmx_YS-JY-22_pool6,28.0,Female,Asian,Healthy,Healthy
CTAACTTCAATGAATG-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0,SeuratProject,-10.592551,84,dmx_YS-JY-22_pool6,1132_1132,4.0,7,cM,NA,1.0,1132_1132:dmx_YS-JY-22_pool6,45.0,Female,European,Managed,SLE
AAGTCTGGTCTACCTC-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,SeuratProject,1.238674,143,dmx_AbFlare-3,FLARE006,3.0,0,cM,NA,0.0,FLARE006:dmx_AbFlare-3,34.0,Female,European,Flare,SLE
GGCTCGATCGTTGACA-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0,SeuratProject,-102.855184,60,dmx_YS-JY-20_pool3,1110_1110,4.0,3,B,B_naive,1.0,1110_1110:dmx_YS-JY-20_pool3,71.0,Female,European,Managed,SLE
ACACCGGCACACAGAG-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0,SeuratProject,-103.921533,91,dmx_YE110,1479_1479,2.0,2,T4,NA,0.0,1479_1479:dmx_YE110,28.0,Female,Asian,Managed,SLE
TCGTAGATCCTTGGTC-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-1-0-0-0-0-0-0-0-0-0-0-0-0-0,SeuratProject,36.016001,107,dmx_YE_8-2,1334_1334,2.0,4,T8,CytoT_GZMH+,1.0,1334_1334:dmx_YE_8-2,52.0,Female,Asian,Managed,SLE


In [13]:
# Clean environment and filterd data
rm(list = ls())
load(file = file.path("object", "00.rda"))
obj <- JoinLayers(obj)
obj[["percent.mt"]] <- PercentageFeatureSet(object = obj, pattern = "^MT-")
obj <- subset(obj, subset = nFeature_RNA > 350 & nFeature_RNA < 5000 & percent.mt < 10)
obj
save(obj, file = file.path("object", "01.rda"))

In [ ]:
save(obj, file = file.path("object", "01.rda"))

In [ ]:
# Normalize data
obj <- SCTransform(obj, ncells = 3000,  variable.features.n = 2000, vst.flavor="v2", method = 'glmGamPoi', conserve.memory = TRUE)
save(obj, file=file.path("object", "02.rda"))

In [ ]:
# Clustering
obj <- RunPCA(obj, npcs = 30, verbose = TRUE)
obj <- RunUMAP(obj, reduction = "pca", dims = 1:20)
obj <- FindNeighbors(obj, reduction = "pca", dims = 1:20)
obj <- FindClusters(obj, resolution = 0.5)
obj
save(obj, file=file.path("object", "03.rda"))

In [ ]:
DimPlot(obj, group.by = "SLE_status")